# what's this
- チーム共有用の日本語メモになります
- せっかくなのでpublic
- このNotebookでは商品価格を眺めることにします

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
sp = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
cl = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
stv = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')

## 下準備
- 追いやすいようにsell_priceとcalendarを結合します
- 全商品点数×全日付だと処理に時間がかかりすぎるので対象の商品を絞れるようにします

In [ ]:
def spcl(item_id):
    result = sp[sp['item_id'].isin(item_id)]
    result = pd.merge(result,cl,on='wm_yr_wk')
    result['date'] = pd.to_datetime(result['date'])
    return result

spcl(['HOBBIES_1_001','HOBBIES_1_002'])

# HOBBIES_1_001の販売価格
- 試しにHOBBIES_1_001の推移を表示します

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
sns.lineplot(data=spcl(['HOBBIES_1_001']),x='date',y='sell_price',hue='store_id',ax=ax)

- 店舗ごとにかなり値段を変えているようです
- ただし大部分の期間で複数店舗が同じ値段なので、他店舗の値段と比較しながら決めているように見えます。
- 2013/9月に販売開始したあとすぐ全体的に値下げしています。売れ行きが悪かったのでしょうか
- TX_1が2015/7月あたりで大幅に値下げしています。セール期間?


# 商品の平均価格と分散と販売日数

In [ ]:
grouped_sp = sp[['item_id','sell_price']].groupby('item_id')
summary_sp = grouped_sp.agg(['mean','max','min','std','var','count'])['sell_price']
summary_sp

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))
for i, column in enumerate(summary_sp.columns):
    sns.distplot(summary_sp[column],ax=axes[i//3,i%3])

- 商品のボリュームゾーンは5ドル以下
  - 高くても20ドル程度
- std
  - 値段の調整は概ね1ドル以下
    - 5ドルくらいの商品ならまぁそんなもんか
- var
  - セールはありそう?、基本的には分散は小さい
- count
  - ほとんどの商品は対象の全期間で販売されている

# 商品のカテゴリ毎に比較 cat_id

In [ ]:
summary_sp1 = summary_sp.copy()
summary_sp1['item_id'] = summary_sp.index
summary_sp1[['cat_id','dept','item']] = summary_sp1['item_id'].str.split('_',expand=True)
summary_sp1['dept_id'] = summary_sp1['cat_id'] + '_' + summary_sp1['dept']
summary_sp1

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))
for i, column in enumerate(['mean', 'max', 'min', 'std', 'var', 'count']):
    sns.distplot(summary_sp1[summary_sp1['cat_id']=='FOODS'][column],ax=axes[i//3,i%3],label='FOODS')
    sns.distplot(summary_sp1[summary_sp1['cat_id']=='HOBBIES'][column],ax=axes[i//3,i%3],label='HOBBIES')
    sns.distplot(summary_sp1[summary_sp1['cat_id']=='HOUSEHOLD'][column],ax=axes[i//3,i%3],label='HOUSEHOLD')

plt.legend()


## カテゴリ毎に描画

## FOODS

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))
for i, column in enumerate(['mean', 'max', 'min', 'std', 'var', 'count']):
    sns.distplot(summary_sp1[summary_sp1['cat_id']=='FOODS'][column],ax=axes[i//3,i%3],label='FOODS')

plt.legend()


## HOBBIES

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))
for i, column in enumerate(['mean', 'max', 'min', 'std', 'var', 'count']):
    sns.distplot(summary_sp1[summary_sp1['cat_id']=='HOBBIES'][column],ax=axes[i//3,i%3],label='HOBBIES')

plt.legend()


## HOUSEHOLD

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))
for i, column in enumerate(['mean', 'max', 'min', 'std', 'var', 'count']):
    sns.distplot(summary_sp1[summary_sp1['cat_id']=='HOUSEHOLD'][column],ax=axes[i//3,i%3],label='HOUSEHOLD')

plt.legend()


- meanを見るとカテゴリによって値段帯の傾向は大きく異る
  - FOODが5ドル以下に集中している
  - HOBBIESは、より低価格帯を中心に高い分散
    - 嗜好品という意味ならタバコとかも入るのだろうか?
  - HOUSEHOLDの特定アイテムの分散が極端に大きくて全体が引っ張られてるように見える
- 困っていること
  - legendをきれいに表示したい

# 商品種別比較 dept_id
- dept_idは7種別にわかれる

In [ ]:
import matplotlib.pyplot as plt
for dept_id in summary_sp1['dept_id'].unique():
    fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))
    for i, column in enumerate(['mean', 'max', 'min', 'std', 'var', 'count']):
            sns.distplot(summary_sp1[summary_sp1['dept_id']==dept_id][column],ax=axes[i//3,i%3])
    fig.suptitle(dept_id, fontsize=20)
    plt.show()



- よくわからないのでカテゴリごとに重ねてみる

In [ ]:
import matplotlib.pyplot as plt
def show_distplot(summary_sp1):
    for dept_id in summary_sp1['dept_id'].unique():
        for i, column in enumerate(['mean', 'max', 'min', 'std', 'var', 'count']):
            sns.distplot(summary_sp1[summary_sp1['dept_id']==dept_id][column],ax=axes[i//3,i%3])
    

for cat_id in summary_sp1['cat_id'].unique():
    fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))
    show_distplot(summary_sp1[summary_sp1['cat_id']==cat_id])
    fig.suptitle(cat_id, fontsize=20)
    plt.show()



- FOODSは価格による種別の差がちょっと見えてこない
- countより、FOOD_2はデータ期間の途中から増えてきた商材に見える
- HOBBIES_1低価格から緩やかに傾斜しているが0.75ドルのあたりに小さなピークがある
- HOBBIES_2は0ドル付近と2ドル付近に2つの価格のピークがある
- HOUSEHOLD_1が低価格帯、HOUSEHOLD_2が中価格帯で分割できそう
- HOUSEHOLDのstdがよく一致している
  - 同じ価格決定のロジックが従っているのかもしれない

